In [1]:
from selenium import webdriver
import time
import re
import requests
import pandas as pd
import numpy as np
import os
import csv
from bs4 import BeautifulSoup

In [5]:
browser= webdriver.Chrome()#声明浏览器
url = 'https://www.icourse163.org/spoc/learn/CUEB-1460898185?tid=1461659481#/learn/forumdetail?pid=1318879010'
browser.get(url)#打开浏览器预设网址
time.sleep(3)
browser.find_element_by_xpath(r'/html/body/div[5]/div[2]/div[1]/div/div[3]/div/div[2]/div/div[2]/div[2]/div/span').click()
time.sleep(3)
#/html/body/div[14]/div[2]/div/div/div/div/div[2]/span
browser.find_element_by_xpath(r'/html/body/div[14]/div[2]/div/div/div/div/div[2]/span').click()
time.sleep(8)
#微信登入
browser.find_element_by_xpath(r'/html/body/div[14]/div[2]/div/div/div/div/div/div[1]/div/div[2]/div[1]/a[2]/span').click()
time.sleep(10)
#点击参加我的课程
browser.find_element_by_xpath(r'/html/body/div[5]/div[2]/div[1]/div/div[3]/div/div[2]/div/div[2]/div[2]/div/span').click()
time.sleep(8)


In [6]:
#输入每个章节的URL地址
#输入每个章节的URL地址
urls = ['https://www.icourse163.org/spoc/learn/CUEB-1460898185?tid=1461659481#/learn/content?type=detail&id=1238139963&sm=1',

'https://www.icourse163.org/spoc/learn/CUEB-1460898185?tid=1461659481#/learn/content?type=detail&id=1238139935&sm=1',

'https://www.icourse163.org/spoc/learn/CUEB-1460898185?tid=1461659481#/learn/content?type=detail&id=1238139948&sm=1',

'https://www.icourse163.org/spoc/learn/CUEB-1460898185?tid=1461659481#/learn/content?type=detail&id=1238139955&sm=1',

'https://www.icourse163.org/spoc/learn/CUEB-1460898185?tid=1461659481#/learn/content?type=detail&id=1238139984&sm=1',

'https://www.icourse163.org/spoc/learn/CUEB-1460898185?tid=1461659481#/learn/content?type=detail&id=1238139972&sm=1',

'https://www.icourse163.org/spoc/learn/CUEB-1460898185?tid=1461659481#/learn/content?type=detail&id=1239464404&sm=1',

'https://www.icourse163.org/spoc/learn/CUEB-1460898185?tid=1461659481#/learn/content?type=detail&id=1238139996&sm=1',

'https://www.icourse163.org/spoc/learn/CUEB-1460898185?tid=1461659481#/learn/content?type=detail&id=1239486337&sm=1'
'https://www.icourse163.org/spoc/learn/CUEB-1460898185?tid=1461659481#/learn/forumdetail?pid=1320060444']

20


In [7]:
#进入每个讨论区章节
for url in urls:
    browser.get(url)
    browser.refresh()
    time.sleep(5)
    #寻找页数的xpath路径
    myPage=browser.find_elements_by_xpath(r'//*[@id="courseLearn-inner-box"]/div/div/div[3]/div[1]/div[2]/div/div/div[4]/div/div[1]/div[2]')
    myPagePatten="style=\"\">(\d*?)</a>"#内容页匹配符
    Pages=re.findall(myPagePatten,myPage[0].get_attribute('innerHTML'))#从源代码中进行匹配
    Page=int(Pages[-1])-1#获得页数

    for i in range(Page): #页数遍历       
        time.sleep(3)
        #获取内容页xpath路径
        contents_main=browser.find_elements_by_xpath("//div[contains(@class,'j-list')]//div[contains(@class,'m-data-lists')]/div[contains(@class,'f-pr')]/div[contains(@class,'m-detailInfoItem f-pr') and contains(@style,'100')]")
        page_nexts=browser.find_elements_by_link_text('下一页')#下一页
        page_next=page_nexts[len(page_nexts)-1]
        myMainPatten="\+\+(\d*?)\-\-(.*?)xx(\d*?)\=\=([\s\S]*)举报"#学号姓名匹配符
        myCommentPagePatten="style=\"\">(\d*?)</a>"#评论匹配符
        mySignPatten="\+\+(\d*?)\-\-(.*?)xx(\d*?)\=\="#签到匹配符
        for content_main in contents_main:
            content_main_text=content_main.text
            if "签到" not in content_main_text:
                content_main_data=re.findall(myMainPatten,content_main_text)     #获取问答区内容
                if not os.path.exists('content_main_result.csv'): 
                    headers = ['Number', 'name','position','content'] #设置表头
                    with open('content_main_result.csv', 'w', newline = '', encoding = 'utf8') as f:  
                        f_csv = csv.writer(f)  #写入文件
                        f_csv.writerow(headers)  
                with open('content_main_result.csv', 'a', newline = '', encoding = 'utf8') as f:
                    f_csv = csv.writer(f)  
                    for i in range(len(content_main_data)):
                        f_csv.writerow(content_main_data[i])
                father_content=content_main.find_element_by_xpath('..')#第一内页
                father_content_text=father_content.get_attribute('innerHTML')#获取源码
                if 'm-commentWrapper' in father_content_text:                      #获取评论区内容
                    comment_content=content_main.find_element_by_xpath("../div[contains(@class,'m-commentWrapper f-pr')]")
                    comment_content_text=comment_content.text
                    comment_content_data=re.findall(myMainPatten,comment_content_text) 
                    if not os.path.exists('content_comment_result.csv'): 
                        headers = ['Number', 'name','position','content'] 
                        with open('content_comment_result.csv', 'w', newline = '', encoding = 'utf8') as f:  
                            f_csv = csv.writer(f)  
                            f_csv.writerow(headers)  
                    with open('content_comment_result.csv', 'a', newline = '', encoding = 'utf8') as f:
                        f_csv = csv.writer(f)  
                        for i in range(len(comment_content_data)):
                            f_csv.writerow(comment_content_data[i])
            else:
                father_content=content_main.find_element_by_xpath('..')
                father_content_text=father_content.get_attribute('innerHTML')
                if 'm-commentWrapper' in father_content_text:                            #获取签到列表
                    sign_content=content_main.find_element_by_xpath("../div[contains(@class,'m-commentWrapper f-pr')]")  
                    comment_page_info=sign_content.get_attribute('innerHTML')#获取签到区源码
                    comment_pages=re.findall(myCommentPagePatten,comment_page_info)#进行匹配
                    if comment_pages:
                        comment_page=int(comment_pages[-1])
                        for i in range(comment_page):
                            time.sleep(1)
                            sign_content_text=sign_content.text
                            sign_content_text_filter=re.findall(mySignPatten,sign_content_text)                        
                            if not os.path.exists('content_sign_result.csv'): 
                                headers = ['Number', 'name','position'] 
                                with open('content_sign_result.csv', 'w', newline = '', encoding = 'utf8') as f:  
                                    f_csv = csv.writer(f)  
                                    f_csv.writerow(headers)  
                            with open('content_sign_result.csv', 'a', newline = '', encoding = 'utf8') as f:
                                f_csv = csv.writer(f)  
                                for i in range(len(sign_content_text_filter)):
                                    f_csv.writerow(sign_content_text_filter[i])
                            sign_content.find_element_by_link_text('下一页').click()
                    else:
                        sign_content_text=sign_content.text
                        sign_content_text_filter=re.findall(mySignPatten,sign_content_text)
                        if not os.path.exists('content_sign_result.csv'): 
                            headers = ['Number', 'name','position'] 
                            with open('content_sign_result.csv', 'w', newline = '', encoding = 'utf8') as f:  
                                f_csv = csv.writer(f)  
                                f_csv.writerow(headers)  
                        with open('content_sign_result.csv', 'a', newline = '', encoding = 'utf8') as f:
                            f_csv = csv.writer(f)  
                            for i in range(len(sign_content_text_filter)):
                                f_csv.writerow(sign_content_text_filter[i])

        page_next.click()
    time.sleep(5)